In [1]:
import pandas as pd
import numpy as np

First, we'll start by importing the New York Times covid data. I made an additional column that equates to the # days after the first US co-vid case for each entry, to make graphing by date easier. 

In [2]:
covid = pd.read_csv('covid2.csv')
covid.rename(columns={'fips':'FIPS'}, inplace=True)

covid['date'] = pd.to_datetime(covid["date"])
start_date = pd.Timestamp("2020-01-21")
covid["daysafter"] = (covid["date"] - start_date).dt.days

covid.head()

print(len(covid))

118343


Next I'll import the social vulnerability index data. This is a really large data set, so I'm immediately filtering out the relevant columns into a more manageable data set.

In [3]:
svi = pd.read_csv('svi-2018.csv')
svi['FIPS'] = svi['FIPS'].astype(float)

svi.head()

socialvuln = svi.filter(items = ['ST', 'COUNTY', 'FIPS', 'AREA_SQMI', 'E_TOTPOP','E_PCI', 'EP_POV', 'EP_NOHSDP', 'EP_AGE65', 'EP_AGE17',
                                 'EP_DISABL', 'EP_SNGPNT', 'EP_MINRTY', 'EP_LIMENG', 'EP_MUNIT', 'EP_MOBILE', 'EP_CROWD', 
                                 'EP_NOVEH', 'EP_GROUPQ', 'RPL_THEME1', 'RPL_THEME2', 'RPL_THEME3', 'RPL_THEME4', 'RPL_THEMES'])

# socialvuln = svi

socialvuln.head()

,ST,COUNTY,FIPS,AREA_SQMI,E_TOTPOP,E_PCI,EP_POV,EP_NOHSDP,EP_AGE65,EP_AGE17,...,EP_MUNIT,EP_MOBILE,EP_CROWD,EP_NOVEH,EP_GROUPQ,RPL_THEME1,RPL_THEME2,RPL_THEME3,RPL_THEME4,RPL_THEMES
0,35,Rio Arriba,35039.0,5860.869195,39307,-999,-999.0,13.8,18.0,23.7,...,0.3,38.8,2.1,6.2,1.7,-999.0000,0.8758,0.9268,0.5409,-999.0000
1,1,Autauga,1001.0,594.443459,55200,29372,15.4,11.3,14.6,24.2,...,3.8,18.4,1.4,5.6,1.0,0.3631,0.5810,0.5947,0.3741,0.4354
2,1,Blount,1009.0,644.830460,57645,22656,14.4,19.8,17.8,23.4,...,0.9,25.2,1.6,4.2,0.9,0.6143,0.3187,0.5915,0.1741,0.4242
3,1,Butler,1013.0,776.838201,20025,20430,23.5,15.4,19.0,22.8,...,1.3,26.2,1.8,7.8,1.6,0.8455,0.9077,0.6447,0.6259,0.8653
4,1,Coffee,1031.0,678.985821,51288,27577,15.1,13.8,16.3,23.7,...,0.9,13.6,1.4,6.0,1.2,0.5373,0.8067,0.7256,0.1923,0.5408


Next I'm importing the economic/unemployment data. Once again, I filter it out into a more manageable set, only taking the columns I'm interested in.

In [4]:
unemp = pd.read_csv('unemployment.csv')

unemp['FIPS'] = unemp['FIPS'].astype(float)
unemp.head()

unemployment = unemp.filter(items=['FIPS', 'State', 'Area_name', 'Rural_urban_continuum_code_2013',
                    'Urban_influence_code_2013', 'Metro_2013', 'Civilian_labor_force_2018',
                    'Employed_2018', 'Unemployed_2018', 'Unemployment_rate_2018', 
                    'Median_Household_Income_2018', 'Med_HH_Income_Percent_of_State_Total_2018'])

unemployment.head()


,FIPS,State,Area_name,Rural_urban_continuum_code_2013,Urban_influence_code_2013,Metro_2013,Civilian_labor_force_2018,Employed_2018,Unemployed_2018,Unemployment_rate_2018,Median_Household_Income_2018,Med_HH_Income_Percent_of_State_Total_2018
0,0.0,US,United States,NaN,NaN,NaN,"161,640,488","155,344,043","6,296,445",3.9,"$61,937",NaN
1,1000.0,AL,Alabama,NaN,NaN,NaN,"2,198,837","2,112,347","86,490",3.9,"$49,881",100.0
2,1001.0,AL,"Autauga County, AL",2.0,2.0,1.0,"25,957","25,015",942,3.6,"$59,338",119.0
3,1003.0,AL,"Baldwin County, AL",3.0,2.0,1.0,"93,849","90,456","3,393",3.6,"$57,588",115.5
4,1005.0,AL,"Barbour County, AL",6.0,6.0,0.0,"8,373","7,940",433,5.2,"$34,382",68.9


Here I merge my unemployment and economic data on FIPs, which is a standardized numerical code for US counties.

In [5]:
merge1 = pd.merge(unemployment, covid, on='FIPS')

merge1['Civilian_labor_force_2018'] = merge1['Civilian_labor_force_2018'].str.replace(',', '').astype(int)
merge1['Employed_2018'] = merge1['Employed_2018'].str.replace(',', '').astype(int)
merge1['Unemployed_2018'] = merge1['Unemployed_2018'].str.replace(',', '').astype(int)
merge1['Median_Household_Income_2018'] = merge1['Median_Household_Income_2018'].str.replace(',', '')
merge1['Median_Household_Income_2018'] = merge1['Median_Household_Income_2018'].str.replace('$', '').astype(int)



merge2 = pd.merge(merge1, socialvuln, on='FIPS')

merge2 = merge2.assign(cases_per_capita = merge2['cases']/merge2['E_TOTPOP'])
merge2 = merge2.assign(cases_per_100k = merge2['cases_per_capita']*100000)
merge2 = merge2.assign(density = merge2['E_TOTPOP']/merge2['AREA_SQMI'])

merge2['daysafter']

0          63
1          64
2          65
3          66
4          67
5          68
6          69
7          70
8          71
9          72
10         73
11         74
12         75
13         76
14         77
15         78
16         79
17         80
18         81
19         82
20         83
21         84
22         85
23         86
24         87
25         88
26         89
27         90
28         91
29         92
         ... 
116941     76
116942     77
116943     78
116944     79
116945     80
116946     81
116947     82
116948     83
116949     84
116950     85
116951     86
116952     87
116953     88
116954     89
116955     90
116956     91
116957     92
116958     93
116959     94
116960     95
116961     96
116962     97
116963     98
116964     99
116965    100
116966    101
116967    102
116968    103
116969    104
116970    105
Name: daysafter, Length: 116971, dtype: int64

In [6]:
#merge2.to_csv('merged.csv', index=False)

In [7]:
latest_date = merge2[merge2['date']=='2020-05-05']

latest_date

,FIPS,State,Area_name,Rural_urban_continuum_code_2013,Urban_influence_code_2013,Metro_2013,Civilian_labor_force_2018,Employed_2018,Unemployed_2018,Unemployment_rate_2018,...,EP_NOVEH,EP_GROUPQ,RPL_THEME1,RPL_THEME2,RPL_THEME3,RPL_THEME4,RPL_THEMES,cases_per_capita,cases_per_100k,density
42,1001.0,AL,"Autauga County, AL",2.0,2.0,1.0,25957,25015,942,3.6,...,5.6,1.0,0.3631,0.5810,0.5947,0.3741,0.4354,0.000960,96.014493,92.859967
95,1003.0,AL,"Baldwin County, AL",3.0,2.0,1.0,93849,90456,3393,3.6,...,3.4,1.4,0.2232,0.1990,0.4358,0.3359,0.2162,0.000908,90.818665,130.901947
128,1005.0,AL,"Barbour County, AL",6.0,6.0,0.0,8373,7940,433,5.2,...,9.2,11.2,0.9780,0.9153,0.8558,0.9889,0.9959,0.001823,182.297727,29.132150
165,1007.0,AL,"Bibb County, AL",1.0,1.0,1.0,8661,8317,344,4.0,...,6.0,9.3,0.7694,0.1203,0.4323,0.7189,0.6003,0.001909,190.882053,36.190214
207,1009.0,AL,"Blount County, AL",1.0,1.0,1.0,25006,24128,878,3.5,...,4.2,0.9,0.6143,0.3187,0.5915,0.1741,0.4242,0.000694,69.390233,89.395591
248,1011.0,AL,"Bullock County, AL",6.0,6.0,0.0,4776,4552,224,4.7,...,11.7,4.2,0.9771,0.4438,0.6749,0.7650,0.8898,0.001739,173.879444,16.621565
290,1013.0,AL,"Butler County, AL",6.0,6.0,0.0,8914,8487,427,4.8,...,7.8,1.6,0.8455,0.9077,0.6447,0.6259,0.8653,0.005993,599.250936,25.777569
339,1015.0,AL,"Calhoun County, AL",3.0,2.0,1.0,45972,43833,2139,4.7,...,5.8,2.7,0.7866,0.8303,0.6507,0.6492,0.8252,0.000990,99.046030,189.972308
387,1017.0,AL,"Chambers County, AL",6.0,5.0,0.0,15276,14679,597,3.9,...,7.3,1.5,0.6901,0.6609,0.4597,0.7943,0.7382,0.008987,898.716963,56.701696
429,1019.0,AL,"Cherokee County, AL",6.0,6.0,0.0,11622,11207,415,3.6,...,4.1,1.1,0.6879,0.4492,0.2751,0.2741,0.4516,0.000658,65.756392,46.706125


In [8]:
print(latest_date['daysafter'].iloc[0]['daysafter'])

IndexError: invalid index to scalar variable.

In [9]:
# h = latest_date.head()

# fip = latest_date['FIPS']

latest_date = latest_date.assign(days_since_first = np.zeros(len(latest_date)))


latest_date.head()




for index, row in latest_date.iterrows():
    i = covid[covid['FIPS'] == row['FIPS']].iloc[0]['daysafter']
    latest_date.set_value(index, 'days_since_first', 105-i)
    
#     fip = h['FIPS']
#     print(fip)
#     first = merge2[merge2['FIPS'] == fip]
#     print(first)
#     firstday = first['daysafter']

# print(len(firstday))


latest_date = latest_date.assign(spread_speed = latest_date['cases_per_100k']/latest_date['days_since_first'])

latest_date.head()

/home/syfgwc/Downloads/usr/local/lib/python3.6/site-packages/ipykernel_launcher.py:15: FutureWarning: set_value is deprecated and will be removed in a future release. Please use .at[] or .iat[] accessors instead
  from ipykernel import kernelapp as app


,FIPS,State,Area_name,Rural_urban_continuum_code_2013,Urban_influence_code_2013,Metro_2013,Civilian_labor_force_2018,Employed_2018,Unemployed_2018,Unemployment_rate_2018,...,RPL_THEME1,RPL_THEME2,RPL_THEME3,RPL_THEME4,RPL_THEMES,cases_per_capita,cases_per_100k,density,days_since_first,spread_speed
42,1001.0,AL,"Autauga County, AL",2.0,2.0,1.0,25957,25015,942,3.6,...,0.3631,0.5810,0.5947,0.3741,0.4354,0.000960,96.014493,92.859967,42.0,2.286059
95,1003.0,AL,"Baldwin County, AL",3.0,2.0,1.0,93849,90456,3393,3.6,...,0.2232,0.1990,0.4358,0.3359,0.2162,0.000908,90.818665,130.901947,52.0,1.746513
128,1005.0,AL,"Barbour County, AL",6.0,6.0,0.0,8373,7940,433,5.2,...,0.9780,0.9153,0.8558,0.9889,0.9959,0.001823,182.297727,29.132150,32.0,5.696804
165,1007.0,AL,"Bibb County, AL",1.0,1.0,1.0,8661,8317,344,4.0,...,0.7694,0.1203,0.4323,0.7189,0.6003,0.001909,190.882053,36.190214,36.0,5.302279
207,1009.0,AL,"Blount County, AL",1.0,1.0,1.0,25006,24128,878,3.5,...,0.6143,0.3187,0.5915,0.1741,0.4242,0.000694,69.390233,89.395591,41.0,1.692445


In [10]:
print(latest_date['daysafter'])

42        105
95        105
128       105
165       105
207       105
248       105
290       105
339       105
387       105
429       105
471       105
512       105
547       105
589       105
631       105
665       105
707       105
742       105
783       105
823       105
864       105
911       105
946       105
988       105
1029      105
1083      105
1123      105
1165      105
1202      105
1246      105
         ... 
115748    105
115796    105
115818    105
115867    105
115923    105
115963    105
115993    105
116046    105
116097    105
116139    105
116163    105
116210    105
116256    105
116295    105
116327    105
116381    105
116421    105
116461    105
116502    105
116552    105
116585    105
116632    105
116659    105
116709    105
116765    105
116804    105
116847    105
116896    105
116930    105
116970    105
Name: daysafter, Length: 2860, dtype: int64


In [11]:
#merge2.to_csv('latestdate.csv', index=False)

In [12]:
merge2.head()

,FIPS,State,Area_name,Rural_urban_continuum_code_2013,Urban_influence_code_2013,Metro_2013,Civilian_labor_force_2018,Employed_2018,Unemployed_2018,Unemployment_rate_2018,...,EP_NOVEH,EP_GROUPQ,RPL_THEME1,RPL_THEME2,RPL_THEME3,RPL_THEME4,RPL_THEMES,cases_per_capita,cases_per_100k,density
0,1001.0,AL,"Autauga County, AL",2.0,2.0,1.0,25957,25015,942,3.6,...,5.6,1.0,0.3631,0.581,0.5947,0.3741,0.4354,0.000018,1.811594,92.859967
1,1001.0,AL,"Autauga County, AL",2.0,2.0,1.0,25957,25015,942,3.6,...,5.6,1.0,0.3631,0.581,0.5947,0.3741,0.4354,0.000072,7.246377,92.859967
2,1001.0,AL,"Autauga County, AL",2.0,2.0,1.0,25957,25015,942,3.6,...,5.6,1.0,0.3631,0.581,0.5947,0.3741,0.4354,0.000109,10.869565,92.859967
3,1001.0,AL,"Autauga County, AL",2.0,2.0,1.0,25957,25015,942,3.6,...,5.6,1.0,0.3631,0.581,0.5947,0.3741,0.4354,0.000109,10.869565,92.859967
4,1001.0,AL,"Autauga County, AL",2.0,2.0,1.0,25957,25015,942,3.6,...,5.6,1.0,0.3631,0.581,0.5947,0.3741,0.4354,0.000109,10.869565,92.859967


In [13]:
len(merge2)

116971

In [14]:
merge2 = merge2.assign(first_day = np.zeros(len(merge2)))
merge2 = merge2.assign(days_since_first = np.zeros(len(merge2)))



#merge2.head()

#temp = merge2.head()

for index, row in merge2.iterrows():
    firstcase = covid[covid['FIPS'] == row['FIPS']].iloc[0]['daysafter']
    
    days = merge2.daysafter.loc[index]
    #print(days)
    merge2.set_value(index, 'first_day', firstcase)
    merge2.set_value(index, 'days_since_first', days - firstcase)


    
merge2.head()

/home/syfgwc/Downloads/usr/local/lib/python3.6/site-packages/ipykernel_launcher.py:15: FutureWarning: set_value is deprecated and will be removed in a future release. Please use .at[] or .iat[] accessors instead
  from ipykernel import kernelapp as app
/home/syfgwc/Downloads/usr/local/lib/python3.6/site-packages/ipykernel_launcher.py:16: FutureWarning: set_value is deprecated and will be removed in a future release. Please use .at[] or .iat[] accessors instead
  app.launch_new_instance()


,FIPS,State,Area_name,Rural_urban_continuum_code_2013,Urban_influence_code_2013,Metro_2013,Civilian_labor_force_2018,Employed_2018,Unemployed_2018,Unemployment_rate_2018,...,RPL_THEME1,RPL_THEME2,RPL_THEME3,RPL_THEME4,RPL_THEMES,cases_per_capita,cases_per_100k,density,first_day,days_since_first
0,1001.0,AL,"Autauga County, AL",2.0,2.0,1.0,25957,25015,942,3.6,...,0.3631,0.581,0.5947,0.3741,0.4354,0.000018,1.811594,92.859967,63.0,0.0
1,1001.0,AL,"Autauga County, AL",2.0,2.0,1.0,25957,25015,942,3.6,...,0.3631,0.581,0.5947,0.3741,0.4354,0.000072,7.246377,92.859967,63.0,1.0
2,1001.0,AL,"Autauga County, AL",2.0,2.0,1.0,25957,25015,942,3.6,...,0.3631,0.581,0.5947,0.3741,0.4354,0.000109,10.869565,92.859967,63.0,2.0
3,1001.0,AL,"Autauga County, AL",2.0,2.0,1.0,25957,25015,942,3.6,...,0.3631,0.581,0.5947,0.3741,0.4354,0.000109,10.869565,92.859967,63.0,3.0
4,1001.0,AL,"Autauga County, AL",2.0,2.0,1.0,25957,25015,942,3.6,...,0.3631,0.581,0.5947,0.3741,0.4354,0.000109,10.869565,92.859967,63.0,4.0


In [18]:
merge2 = merge2.assign(spread_speed = merge2['cases_per_100k']/merge2['days_since_first'])
merge2['spread_speed'].unique()

from numpy import inf
merge2['spread_speed'] = merge2['spread_speed'].replace(inf,1)

merge2['spread_speed'].loc[0]

merge2.filter(items=['county','E_TOTPOP','cases', 'first_day', 'cases_per_100k', 'days_since_first', 'spread_speed']).head()

,county,E_TOTPOP,cases,first_day,cases_per_100k,days_since_first,spread_speed
0,Autauga,55200,1,63.0,1.811594,0.0,1.000000
1,Autauga,55200,4,63.0,7.246377,1.0,7.246377
2,Autauga,55200,6,63.0,10.869565,2.0,5.434783
3,Autauga,55200,6,63.0,10.869565,3.0,3.623188
4,Autauga,55200,6,63.0,10.869565,4.0,2.717391


In [19]:
merge2.to_csv('finaldataset.csv', index=False)